In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

/opt/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
def xavier_initializer(n_inputs, n_outputs, uniform = True):
    if uniform:
        init_range = tf.sqrt(6.0/ (n_inputs + n_outputs))
        return tf.random_uniform_initializer(-init_range, init_range)

    else:
        stddev = tf.sqrt(3.0 / (n_inputs + n_outputs))
        return tf.truncated_normal_initializer(stddev=stddev)

In [11]:
class NN:
    def __init__(self, sess, name, input_dim=784, output_dim=10, hidden_dims=[500], activation_fn=tf.nn.relu, optimizer=tf.train.AdamOptimizer, lr=0.01):
        self.sess = sess
        with tf.variable_scope(name):
            self.X = tf.placeholder(tf.float32, [None, input_dim], name='X')
            self.Y = tf.placeholder(tf.float32, [None, output_dim], name = 'Y')
            self.mode = tf.placeholder(tf.bool, name='train_mode')
            
            current_dim = input_dim
            current_layer = self.X
            
            for i, h_dim in enumerate(hidden_dims):
                with tf.variable_scope('layer{}'.format(i)):
                    W = tf.get_variable(shape=[current_dim, h_dim], name='weight', initializer=xavier_initializer(current_dim, h_dim))
                    b = tf.Variable(tf.zeros([h_dim]), name='bias')
                    layer = tf.matmul(current_layer, W) + b    
                    current_layer = activation_fn(layer)
                    current_dim = h_dim
            
            with tf.variable_scope('final_layer'):
                W = tf.get_variable(shape=[current_dim, output_dim], name='weight', initializer=xavier_initializer(current_dim, output_dim))
                b = tf.Variable(tf.zeros([output_dim]), name='bias')
                current_layer = tf.matmul(current_layer, W) + b
            
            self.loss = tf.nn.softmax_cross_entropy_with_logits(logits=current_layer, labels = self.Y)
            self.loss = tf.reduce_mean(self.loss, name='loss')
            self.train_op = optimizer(lr).minimize(self.loss)        
        
            softmax = tf.nn.softmax(current_layer, name='softmax')
            self.accuracy = tf.equal(tf.argmax(softmax, 1), tf.argmax(self.Y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(self.accuracy, tf.float32))
            
            sess.run(tf.global_variables_initializer())
    
    
    def train(self,X,Y):
        feed = {
            self.X:X,
            self.Y:Y,
            self.mode: True
        }
        return self.sess.run([self.train_op, self.loss], feed_dict=feed)
    
    
    def evaluate(self, X, Y, batch_size=None):
        feed = {
            self.X: X,
            self.Y: Y,
            self.mode: False
        }
        return self.sess.run([self.loss, self.accuracy], feed_dict=feed)

In [22]:
TRAINING_EPOCHS = 10
BATCH_SIZE = 500
N = 55000

In [23]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = NN(sess,'first_model')

In [24]:
for epoch in range(TRAINING_EPOCHS):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / BATCH_SIZE)
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(BATCH_SIZE)
        _, c = model.train(batch_xs, batch_ys)
        avg_cost += c / total_batch

    print('Epoch: ' + str((epoch + 1)) + ' cost: ' + str((avg_cost)))
    l, p = model.evaluate(mnist.test.images, mnist.test.labels)
    
    print('Precision: ' + str(p))

print('Learning Finished!')


Epoch: 1 cost: 0.296566475589
Precision: 0.9649
Epoch: 2 cost: 0.0932131653821
Precision: 0.9701
Epoch: 3 cost: 0.0609748106958
Precision: 0.976
Epoch: 4 cost: 0.0440022152425
Precision: 0.9741
Epoch: 5 cost: 0.0356760977937
Precision: 0.9763
Epoch: 6 cost: 0.0285113705788
Precision: 0.9762
Epoch: 7 cost: 0.0238379813773
Precision: 0.978
Epoch: 8 cost: 0.0204468847803
Precision: 0.9777
Epoch: 9 cost: 0.0193043943198
Precision: 0.974
Epoch: 10 cost: 0.0202411584006
Precision: 0.9782
Learning Finished!
